In [89]:
from __future__ import print_function

import os
import sys
import numpy as np

from nltk.stem import PorterStemmer

from sklearn.model_selection import train_test_split

from keras.preprocessing.text import Tokenizer
from keras.preprocessing.sequence import pad_sequences
from keras import layers, Sequential
from keras.utils import to_categorical
from keras.layers import Dense, Input, GlobalMaxPooling1D
from keras.layers import Conv1D, MaxPooling1D, Embedding
from keras.models import Model
from keras.initializers import Constant

import pandas as pd

In [54]:
BASE_DIR = ''
GLOVE_DIR = BASE_DIR
TEXT_DATA_DIR = os.path.join(BASE_DIR, '20_newsgroup')
MAX_SEQUENCE_LENGTH = 200
MAX_NUM_WORDS = 1000
EMBEDDING_DIM = 100
VALIDATION_SPLIT = 0.2

In [28]:
print('Indexing word vectors.')

embeddings_index = {}
with open(os.path.join(GLOVE_DIR, 'glove.6B.100d.txt')) as f:
    for line in f:
        word, coefs = line.split(maxsplit=1)
        coefs = np.fromstring(coefs, 'f', sep=' ')
        embeddings_index[word] = coefs

print('Found %s word vectors.' % len(embeddings_index))

Indexing word vectors.
Found 400000 word vectors.


In [75]:
dataset = pd.read_csv('aclImdb/train.csv')
texts = dataset['text'].values
labels = dataset['target'].values
labels_index = {
    1: 1,
    0: 0,
}

In [76]:
tokenizer = Tokenizer(num_words=MAX_NUM_WORDS)
tokenizer.fit_on_texts(texts)
sequences = tokenizer.texts_to_sequences(texts)

word_index = tokenizer.word_index
print('Found %s unique tokens.' % len(word_index))

Found 19390 unique tokens.


In [77]:
data = pad_sequences(sequences, maxlen=MAX_SEQUENCE_LENGTH)

labels = to_categorical(np.asarray(labels))
print('Shape of data tensor:', data.shape)
print('Shape of label tensor:', labels.shape)

Shape of data tensor: (1000, 200)
Shape of label tensor: (1000, 2)


In [114]:
X_train, X_test, y_train, y_test \
    = train_test_split(data, labels, test_size=0.2, random_state=42)

X_train, X_val, y_train, y_val \
    = train_test_split(X_train, y_train, test_size=0.2, random_state=42)

In [83]:
num_words = min(MAX_NUM_WORDS, len(word_index) + 1)
embedding_matrix = np.zeros(shape=(num_words, EMBEDDING_DIM))
for word, i in word_index.items():
    if i >= MAX_NUM_WORDS:
        continue
    embedding_vector = embeddings_index.get(word)
    if embedding_vector is not None:
        embedding_matrix[i] = embedding_vector

In [88]:
embedding_layer = layers.Embedding(
    num_words,
    EMBEDDING_DIM,
    embeddings_initializer=Constant(embedding_matrix),
    input_length=MAX_SEQUENCE_LENGTH,
    trainable=False
)

In [111]:
model = Sequential([
    layers.Embedding(
        num_words,
        EMBEDDING_DIM,
        input_shape=(MAX_SEQUENCE_LENGTH,),
        embeddings_initializer=Constant(embedding_matrix),
        input_length=MAX_SEQUENCE_LENGTH,
        trainable=False
    ),
    layers.Conv1D(128, 5, activation='relu'),
    layers.MaxPooling1D(5),
    layers.Conv1D(128, 5, activation='relu'),
    layers.MaxPooling1D(5),
    layers.Conv1D(128, 5, activation='relu'),
    layers.GlobalMaxPooling1D(),
    layers.Dense(128, activation='relu'),
    layers.Dense(len(labels_index), activation='softmax')
])

In [112]:
model.compile(loss='categorical_crossentropy',
              optimizer='rmsprop',
              metrics=['acc'])

In [119]:
model.fit(X_train, y_train,
          batch_size=128,
          epochs=10,
          verbose=1,
          validation_data=(X_val, y_val))

Train on 640 samples, validate on 160 samples
Epoch 1/10
640/640 [==============================] - 1s 958us/step - loss: 0.6604 - acc: 0.6203 - val_loss: 0.7140 - val_acc: 0.5000
Epoch 2/10
640/640 [==============================] - 1s 1ms/step - loss: 0.5511 - acc: 0.7766 - val_loss: 1.0155 - val_acc: 0.4625
Epoch 3/10
640/640 [==============================] - 1s 935us/step - loss: 0.6175 - acc: 0.6359 - val_loss: 0.7014 - val_acc: 0.5813
Epoch 4/10
640/640 [==============================] - 1s 988us/step - loss: 0.8249 - acc: 0.6719 - val_loss: 0.6976 - val_acc: 0.5375
Epoch 5/10
640/640 [==============================] - 1s 950us/step - loss: 0.4779 - acc: 0.8391 - val_loss: 1.7556 - val_acc: 0.5250
Epoch 6/10
640/640 [==============================] - 1s 955us/step - loss: 0.6643 - acc: 0.7953 - val_loss: 0.7221 - val_acc: 0.5375
Epoch 7/10
640/640 [==============================] - 1s 955us/step - loss: 0.4556 - acc: 0.8078 - val_loss: 0.7780 - val_acc: 0.5375
Epoch 8/10
640/640

In [124]:
loss, acc = model.evaluate(X_test, y_test)
print('Accuracy is', acc)

200/200 [==============================] - 0s 492us/step
Accuracy is 0.47999998927116394
